# Prática: Classificação com Redes Neurais em PyTorch

## Introdução

Este notebook é um exercício prático para a construção, treinamento e avaliação de uma rede neural para um problema de classificação. As células de texto (Markdown) fornecerão o embasamento teórico, e sua tarefa será implementar a lógica correspondente nas células de código subsequentes.

O objetivo é solidificar o entendimento sobre o pipeline de um projeto em PyTorch, desde a manipulação de dados até a avaliação do modelo.

In [14]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

## 1. Geração e Visualização do Dataset

Para que a aplicação de uma rede neural seja justificada, o problema de classificação não deve ser linearmente separável. Utilizaremos o `numpy` para gerar tal dataset.

Para fins de visualização e para aumentar a dimensionalidade, uma terceira feature será artificialmente criada como uma combinação não-linear das duas features originais. Por fim, os dados serão normalizados, o que é uma prática recomendada que auxilia na estabilidade e velocidade do treinamento de redes neurais.

In [15]:
# Geração e preparação do dataset
from sklearn.preprocessing import StandardScaler

def generate_spiral_data(n_samples_per_class=1000, n_turns=3, noise=0.3):
    """
    Gera um dataset de duas espirais 3D entrelaçadas.
    """
    # Geração dos pontos da primeira espiral (Classe 0)
    t = np.linspace(0, n_turns * 2 * np.pi, n_samples_per_class)
    x1 = t * np.cos(t)
    y1 = t * np.sin(t)
    z1 = t

    # Adição de ruído gaussiano
    X1 = np.vstack((x1, y1, z1)).T
    X1 += noise * np.random.randn(*X1.shape)
    y1 = np.zeros(n_samples_per_class)

    # Geração dos pontos da segunda espiral (Classe 1), defasada em 180 graus
    x2 = t * np.cos(t + np.pi)
    y2 = t * np.sin(t + np.pi)
    z2 = t

    # Adição de ruído gaussiano
    X2 = np.vstack((x2, y2, z2)).T
    X2 += noise * np.random.randn(*X2.shape)
    y2 = np.ones(n_samples_per_class)

    # Combinação e embaralhamento dos dados
    X = np.vstack((X1, X2))
    y = np.concatenate((y1, y2))

    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)

    X = X[indices]
    y = y[indices]

    return X, y

# Geração dos dados
X, y = generate_spiral_data()

# Normalização dos dados, uma prática padrão
scaler = StandardScaler()
X = scaler.fit_transform(X)

print(f"Shape de X (features): {X.shape}")
print(f"Shape de y (labels): {y.shape}")

Shape de X (features): (2000, 3)
Shape de y (labels): (2000,)


### Visualização 3D dos Dados

A visualização do dataset em um espaço tridimensional nos permite obter uma intuição sobre a complexidade da fronteira de decisão que o modelo precisará aprender para separar as classes.

In [16]:
fig = px.scatter_3d(
    x=X[:, 0],
    y=X[:, 1],
    z=X[:, 2],
    color=y,
    color_continuous_scale=px.colors.qualitative.Vivid,
    title="Dataset Sintético 3D para Classificação"
)
fig.update_traces(marker=dict(size=3))
fig.show()

## 2. Preparação dos Dados para o PyTorch

Nesta seção, você irá encapsular os dados NumPy em classes `Dataset` e `DataLoader` do PyTorch, que são abstrações fundamentais para o carregamento e a iteração sobre os dados de forma eficiente durante o treinamento.

### 2.1. A Classe `Dataset`
É necessário criar uma classe que herde de `torch.utils.data.Dataset`. Esta classe customizada deve implementar três métodos:
- `__init__(self, features, labels)`: O construtor, onde os dados são recebidos. É aqui que os arrays NumPy devem ser convertidos para tensores do PyTorch. As features (X) devem ser do tipo `torch.float32` e os rótulos (y) do tipo `torch.long`.
- `__len__(self)`: Método que retorna o número total de amostras no dataset.
- `__getitem__(self, idx)`: Método que permite o acesso a uma amostra específica do dataset através de um índice `idx`. Ele deve retornar um par `(feature, label)`.

In [28]:
import torch
from torch.utils.data import Dataset
import numpy as np

class SeuDataset(Dataset):

    def __init__(self, features: np.ndarray, labels: np.ndarray):

        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)


    def __len__(self) -> int:
        return len(self.features)

    def __getitem__(self, idx: int) -> tuple[torch.Tensor, torch.Tensor]:
        return self.features[idx], self.labels[idx]

### 2.2. Divisão dos Dados e `DataLoader`
Com a classe `Dataset` definida, o próximo passo é:
1.  Dividir os arrays `X` e `y` em conjuntos de treinamento e teste utilizando `train_test_split`.
2.  Instanciar a sua classe Dataset para cada um desses conjuntos.
3.  Criar instâncias de `DataLoader` para os dois datasets. O `DataLoader` é um iterador que agrupa os dados em mini-lotes (`mini-batches`), com a opção de embaralhá-los a cada época, uma prática essencial para o conjunto de treino.

In [29]:
from sklearn.model_selection import train_test_split

# X_train, X_test, ...
# train_dataset = ...
# test_dataset = ...

# --- Criando dados NumPy do zero ---
print("--- Criando dados fictícios ---")
X_geral = np.random.rand(1000, 20).astype('float32') # 1000 amostras, 20 features
y_geral = np.random.randint(0, 4, size=1000)        # 1000 rótulos para 4 classes

# --- Preenchendo a primeira parte do seu exercício ---
print("--- Dividindo dados em treino e teste ---")
X_train, X_test, y_train, y_test = train_test_split(
    X_geral, y_geral, test_size=0.2, random_state=42, stratify=y_geral
)

print(f"Shape de X_train: {X_train.shape}")
print(f"Shape de X_test: {X_test.shape}")

print("\n--- Instanciando os Datasets ---")
# Instanciar a classe para o conjunto de treinamento
train_dataset = SeuDataset(features=X_train, labels=y_train)

# Instanciar a classe para o conjunto de teste
test_dataset = SeuDataset(features=X_test, labels=y_test)

print(f"Tamanho do train_dataset: {len(train_dataset)} amostras")
print(f"Tamanho do test_dataset: {len(test_dataset)} amostras")


--- Criando dados fictícios ---
--- Dividindo dados em treino e teste ---
Shape de X_train: (800, 20)
Shape de X_test: (200, 20)

--- Instanciando os Datasets ---
Tamanho do train_dataset: 800 amostras
Tamanho do test_dataset: 200 amostras


In [30]:
from torch.utils.data import DataLoader

# batch_size = ...
# train_loader = ...
# test_loader = ...

# --- Importação necessária para esta célula ---
from torch.utils.data import DataLoader

# --- Preenchendo a segunda parte do seu exercício ---
print("--- Criando os DataLoaders ---")

# Definir o tamanho do lote (batch size)
batch_size = 32

# Criar o DataLoader para o conjunto de treino (com shuffle=True)
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch_size,
                          shuffle=True)

# Criar o DataLoader para o conjunto de teste (com shuffle=False)
test_loader = DataLoader(dataset=test_dataset,
                         batch_size=batch_size,
                         shuffle=False)


# --- Verificação ---
print(f"DataLoaders criados com batch_size={batch_size}")
# Pega um lote de dados de treino para inspecionar
features_batch, labels_batch = next(iter(train_loader))
print(f"Shape de um lote de treino (features): {features_batch.shape}")
print("\nCélula 2 executada com sucesso! As variáveis 'train_loader' e 'test_loader' estão prontas.")

--- Criando os DataLoaders ---
DataLoaders criados com batch_size=32
Shape de um lote de treino (features): torch.Size([32, 20])

Célula 2 executada com sucesso! As variáveis 'train_loader' e 'test_loader' estão prontas.


## 3. Arquitetura da Rede Neural

A arquitetura do modelo será composta por camadas ocultas e uma camada de saída.
- As camadas ocultas são responsáveis por aprender representações complexas dos dados e podem ser construídas com `nn.Linear` e `nn.ReLU`.
- A **camada de saída** deve ter **apenas 1 neurônio**.
- Após a última camada linear, deve ser aplicada uma função de ativação `nn.Sigmoid()`.

A saída do modelo será um único valor entre 0 e 1 para cada amostra de entrada, que pode ser interpretado como a probabilidade da amostra pertencer à classe 1.
$$ \hat{y} = \sigma(W_{\text{out}} \cdot a_{\text{hidden}} + b_{\text{out}}) $$
Onde $a_{\text{hidden}}$ é a ativação da última camada oculta e $\sigma$ é a função Sigmoid.

In [20]:
import torch.nn as nn

# class ...(nn.Module):
    # Implemente os métodos __init__ e forward

# model = ...
# print(model)

## 4. Função de Custo e Otimizador

A **função de custo** para este problema será a `nn.BCELoss` (*Binary Cross-Entropy Loss*). Esta função mede o erro entre a probabilidade prevista pelo modelo e o rótulo verdadeiro (0 ou 1), sendo a escolha canônica para classificação binária.

Para o **otimizador**, utilizaremos o `torch.optim.SGD`, que implementa o algoritmo de descida do gradiente estocástico. Sua principal configuração é a **taxa de aprendizado** (`learning_rate`).

In [21]:
# Definição da loss e do otimizador

# learning_rate = ...
# loss_fn = ...
# optimizer = ...

## 5. Loop de Treinamento

O loop de treinamento segue uma estrutura de 5 passos, iterando sobre os dados por um número definido de épocas. Para cada lote de dados, o ciclo é:

1.  **Forward Pass**: Propagar os dados de entrada pelo modelo para obter as predições.
2.  **Cálculo da Perda**: Calcular a perda comparando as predições com os rótulos verdadeiros.
3.  **Zerar Gradientes**: Limpar os gradientes da iteração anterior (`optimizer.zero_grad()`).
4.  **Backward Pass (Backpropagation)**: Calcular os gradientes da perda em relação a cada parâmetro (`loss.backward()`).
5.  **Atualização dos Pesos**: Atualizar os pesos do modelo usando o otimizador (`optimizer.step()`).

In [22]:
# Implementação do loop de treinamento

# num_epochs = ...
# for epoch in range(num_epochs):
#     ...

## 6. Visualização e Avaliação do Desempenho

### Curvas de Aprendizagem
Após o treinamento, é fundamental analisar as curvas de aprendizagem. A plotagem da perda e da acurácia ao longo das épocas nos permite diagnosticar se o modelo aprendeu corretamente e se há sinais de problemas como *overfitting* ou *underfitting*.

In [23]:
# Plot das curvas de loss e acurácia

### Avaliação Final no Conjunto de Teste
A avaliação final deve ser feita no conjunto de teste, que o modelo não viu durante o treinamento. Isso fornece uma estimativa imparcial de sua capacidade de generalização.

Para a fase de avaliação (inferência), é importante colocar o modelo em modo de avaliação com `model.eval()` e realizar os cálculos dentro de um bloco `with torch.no_grad()` para desativar o cálculo de gradientes.

In [24]:
# Avaliação no dataset de teste

# model.eval()
# with torch.no_grad():
#    ...
#    # predicted = (outputs > 0.5).float()

In [25]:
# fig = px.scatter_3d(
#     x=...,
#     y=...,
#     z=...,
#     color=...,
#     color_continuous_scale=px.colors.qualitative.Vivid,
#     title="Predições no Conjunto de Testes"
# )
# fig.update_traces(marker=dict(size=3))
# fig.show()